<a href="https://colab.research.google.com/github/unique-subedi/gene-expression/blob/main/Vinod_gene_expression_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import datetime
import time
import math
import numpy as np
from numpy import linalg as LA
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt

In [3]:
!pip install pyreadr
import pyreadr
urllib.request.urlretrieve("https://raw.githubusercontent.com/unique-subedi/gene-expression/main/data/brain.rda", "brain.rda")
brain = pyreadr.read_r("brain.rda")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 363 kB 4.0 MB/s 


In [4]:
expression = pd.DataFrame(brain["expression"])
genes = pd.DataFrame(brain["genes"])
samples = pd.DataFrame(brain["samples"])

In [5]:
genes.head()

,sym,chrom
rownames,,
1000_at,MAPK3,16
1001_at,TIE1,1
1002_f_at,CYP2C19,10
1003_s_at,CXCR5,11
1004_at,CXCR5,11


In [23]:
express_samp = pd.concat([expression, samples], axis=1)

# for each gene, need to output three numbers. One for each tissue, which summarizes the difference between average male expression and average female expression
# the genes are the columns. For each gene we need three numbers one for each tissue. Select Michigan, Select, Tissue, Select Male, avearge for each gene. 
# Select Michigan, Select Tissue, Select Female, average for each gene. 

express_samp.head()
express_samp_UM = express_samp[express_samp.lab == "Michigan"]
# print(express_samp_UM.head())
for tissue in express_samp_UM["region"].unique():
  print(tissue)
  df_male = express_samp_UM[(express_samp_UM.region == tissue) & (express_samp_UM.sex == "male")]
  df_female = express_samp_UM[(express_samp_UM.region == tissue) & (express_samp_UM.sex == "female")]

  male_mean = expression.loc[df_male.index].mean()
  female_mean = expression.loc[df_female.index].mean()

  diff = np.abs(male_mean - female_mean)
  diff = diff.sort_values(ascending=False)
  top_10 = diff.iloc[:10]

  print(top_10)
  print("")





A.C. cortex
41214_at      0.965024
41158_at      0.950039
37743_at      0.631292
38355_at      0.631031
41123_s_at    0.608219
347_s_at      0.599001
40567_at      0.594510
33117_r_at    0.590882
33676_at      0.572482
32538_at      0.564630
dtype: float64

cerebellum
31687_f_at    0.790109
31525_s_at    0.772914
41158_at      0.772398
32052_at      0.772187
41214_at      0.763528
35817_at      0.718232
34342_s_at    0.579688
38355_at      0.567901
40185_at      0.548879
2092_s_at     0.493642
dtype: float64

D.L.P.F. cortex
41214_at                     1.458640
38355_at                     0.804372
32052_at                     0.486683
AFFX-HUMGAPDH/M33197_5_at    0.466037
34272_at                     0.438816
1161_at                      0.430753
38462_at                     0.421535
32652_g_at                   0.419196
37183_at                     0.415238
AFFX-HUMGAPDH/M33197_M_at    0.415042
dtype: float64

